# 영화추천 시스템 - ver.2
- 코사인 유사도 활용
- TfidfVectorizer 사용
- 영화의 줄거리, 주연 배우, 감독 

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
movie = pd.read_csv('data/movies/movies_metadata.csv', low_memory=False)
info = pd.read_csv('data/movies/credits.csv')
info.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [3]:
movie.shape, info.shape

((45466, 24), (45476, 3))

- 데이터 전처리

In [4]:
df = movie[['id','title','overview']]
df.head(3)

,id,title,overview
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...


In [5]:
# 결측치 제거
df.dropna(inplace=True)
df.shape

(44506, 3)

In [6]:
df.drop_duplicates(subset=['title'], inplace=True)
df.shape

(41371, 3)

In [7]:
df.id = df.id.astype(int)
df.head(3)

,id,title,overview
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...


In [8]:
info.id = info.id.astype(int)
df = pd.merge(df, info)
df.head(3)

,id,title,overview,cast,crew
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."


In [9]:
df.shape

(41411, 5)

In [10]:
df = df.head(20000)

In [11]:
df.set_index('id', inplace=True)
df.head()

,title,overview,cast,crew
id,,,,
862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
11862,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."


- cast 처리

In [12]:
df.cast[862]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [13]:
from ast import literal_eval
df.cast = df.cast.apply(literal_eval)
df.head(3)

,title,overview,cast,crew
id,,,,
862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."


In [14]:
df.cast[862][0]['name']

'Tom Hanks'

In [15]:
import re
actors = ['Tom Hanks', 'Tim Allen']
list(map(lambda x: re.sub(' ', '', x), actors))

['TomHanks', 'TimAllen']

In [16]:
def get_3cast(x):
    cast = []
    for item in x:
        if item['name'] not in cast:
            cast.append(item['name'])
    cast = list(map(lambda s: re.sub(' ', '', s), cast))
    cast = cast if len(cast) <= 3 else cast[:3]
    return ' '.join(cast)

In [17]:
df['cast3'] = df.cast.apply(get_3cast)
df.head(3)

,title,overview,cast,crew,cast3
id,,,,,
862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",TomHanks TimAllen DonRickles
8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",RobinWilliams JonathanHyde KirstenDunst
15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",WalterMatthau JackLemmon Ann-Margret


- 감독

In [18]:
df.crew = df.crew.apply(literal_eval)
df.crew[862][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [19]:
def get_director(x):
    for item in x:
        if item['job'] == 'Director':
            return item['name'].replace(' ', '')
    return ''

In [20]:
df['director'] = df.crew.apply(get_director)
df.head(3)

,title,overview,cast,crew,cast3,director
id,,,,,,
862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",TomHanks TimAllen DonRickles,JohnLasseter
8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",RobinWilliams JonathanHyde KirstenDunst,JoeJohnston
15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",WalterMatthau JackLemmon Ann-Margret,HowardDeutch


In [21]:
df.reset_index(inplace=True)
df.head(3)

,id,title,overview,cast,crew,cast3,director
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",TomHanks TimAllen DonRickles,JohnLasseter
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",RobinWilliams JonathanHyde KirstenDunst,JoeJohnston
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",WalterMatthau JackLemmon Ann-Margret,HowardDeutch


- DTM 변환

In [22]:
df['total'] = df.overview + ' ' + df.director + ' ' + df.cast3
df.total[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. JohnLasseter TomHanks TimAllen DonRickles"

In [23]:
# 감독 이름을 3번 사용, 감독을 강조
df['total'] = df.overview + (' ' + df.director) * 3 + ' ' + df.cast3
df.total[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. JohnLasseter JohnLasseter JohnLasseter TomHanks TimAllen DonRickles"

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(stop_words='english')
dtm = tvect.fit_transform(df.total)
dtm.shape

(20000, 77421)

In [25]:
indices = pd.Series(df.index, index=df.title)

In [26]:
# 코사인 유사도
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(dtm, dtm)

In [27]:
def get_recommendation(title, cos_sim=cosine_sim):
    index = indices[title]
    sim_scores = pd.Series(cosine_sim[index])
    movie_indices = sim_scores.sort_values(ascending=False).head(11).tail(10).index
    return df.title.iloc[movie_indices]

In [28]:
get_recommendation('Toy Story', cosine_sim)

2949                Toy Story 2
14728               Toy Story 3
18173                   Tin Toy
18267               Knick Knack
16580                    Cars 2
2216               A Bug's Life
10337                  Luxo Jr.
18223               Red's Dream
9996     The 40 Year Old Virgin
10636                      Cars
Name: title, dtype: object